In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import time
import imp
import os
import os, os.path
import datetime as dt
import dateutil.parser as dp
import dill

%matplotlib inline

In [2]:
# Create lists of data files (3 groups: regular attributes, GEO, and Seller composition)

#data_directory = "../30y fixed rate data/fnm/conforming"
data_directory = "../30y fixed rate data/fnm/jumbos"

# pools attributes 
pools_attributes_files = list()

for root, dirs, files in os.walk(data_directory):
    for name in files:
        file_name = os.path.join(root, name)
        if 'pools attributes' in file_name:
            pools_attributes_files.append(file_name)
        
# pools geographical composition        
pools_geo_pct_files = list()

for root, dirs, files in os.walk(data_directory):
    for name in files:
        file_name = os.path.join(root, name)
        if 'geo pct' in file_name:
            pools_geo_pct_files.append(os.path.join(root, name))  
        
# pools seller composition        
pools_seller_pct_files = list()

for root, dirs, files in os.walk(data_directory):
    for name in files:
        file_name = os.path.join(root, name)
        if 'seller pct' in file_name:
            pools_seller_pct_files.append(os.path.join(root, name)) 
        
print("# of attr files: ",len(pools_attributes_files))
print("# of geo pct files: ",len(pools_geo_pct_files))
print("# of seller pct files: ",len(pools_seller_pct_files))

# of attr files:  130
# of geo pct files:  130
# of seller pct files:  130


In [3]:
# Read in all files with regular pool attributes
attr_df = pd.DataFrame()
for file in pools_attributes_files:
    one_month_issue_pools = pd.read_csv(file)
    if len(attr_df)==0:
        attr_df = one_month_issue_pools
    else:
        if all(attr_df.columns == one_month_issue_pools.columns):
            attr_df = pd.concat([attr_df,one_month_issue_pools])
        else:
            print(f'columns missmatch for {file}')

    #print('Done loading ' + file)

In [4]:
attr_df.columns

Index(['poolno', 'asofdate', 'cusip', 'Prefix', 'spread', 'Cpr1', 'Cpr3',
       'Cpr6', 'Cpr12', 'Cpr24', 'CprLife', 'SMM', 'DayCount', 'OBal', 'CBal',
       'pbal', 'paydown', 'Prepay', 'factor', 'OCoupon', 'Coupon', 'OWac',
       'Wac', 'Wam', 'Age', 'aols', 'waols', 'ONLoans', 'cnloans', 'PCNLoans',
       'PPNLoans', 'OSato', 'CSato', 'oltv', 'cltv', '%cltv_80', '%cltv_105',
       '%cltv_125', '%ccltv_80', '%ccltv_105', '%ccltv_125', 'fico',
       '%FedHold', '%CMOHold', 'dti', 'codti', '%CashWindow', '%Majors',
       'ocltv', 'ccltv', 'PurpPct_purchase', 'PurpPct_refi',
       'PctChannel_Broker', 'PctChannel_Corr', 'PctChannel_Retail',
       'OccPct_investor', 'OccPct_owner', 'PropUnitsPct_2-4', 'Label'],
      dtype='object')

In [5]:
print(f'attr_df.shape = {attr_df.shape}')

attr_df.shape = (225957, 59)


In [6]:
# Read in all files with pools GEO info
geo_df = pd.DataFrame()
for file in pools_geo_pct_files:
    one_month_issue_pools = pd.read_csv(file)
    if len(geo_df)==0:
        geo_df = one_month_issue_pools
    else:
        if all(geo_df.columns == one_month_issue_pools.columns):
            geo_df = pd.concat([geo_df,one_month_issue_pools])
        else:
            print(f'columns missmatch for {file}')

    #print('Done loading ' + file)

In [7]:
geo_df.columns

Index(['poolno', 'asofdate', 'cusip', 'StatePct_AK', 'StatePct_AL',
       'StatePct_AR', 'StatePct_AZ', 'StatePct_CA', 'StatePct_CO',
       'StatePct_CT', 'StatePct_DC', 'StatePct_DE', 'StatePct_FL',
       'StatePct_GA', 'StatePct_GU', 'StatePct_HI', 'StatePct_IA',
       'StatePct_ID', 'StatePct_IL', 'StatePct_IN', 'StatePct_KS',
       'StatePct_KY', 'StatePct_LA', 'StatePct_MA', 'StatePct_MD',
       'StatePct_ME', 'StatePct_MI', 'StatePct_MN', 'StatePct_MO',
       'StatePct_MS', 'StatePct_MT', 'StatePct_NC', 'StatePct_ND',
       'StatePct_NE', 'StatePct_NH', 'StatePct_NJ', 'StatePct_NM',
       'StatePct_NV', 'StatePct_NY', 'StatePct_OH', 'StatePct_OK',
       'StatePct_OR', 'StatePct_PA', 'StatePct_PR', 'StatePct_RI',
       'StatePct_SC', 'StatePct_SD', 'StatePct_TN', 'StatePct_TX',
       'StatePct_UT', 'StatePct_VA', 'StatePct_VI', 'StatePct_VT',
       'StatePct_WA', 'StatePct_WI', 'StatePct_WV', 'StatePct_WY', 'Label'],
      dtype='object')

In [8]:
print(f'geo_df.shape = {geo_df.shape}')

geo_df.shape = (225957, 58)


In [9]:
# Read in all files with pools Seller info
seller_df = pd.DataFrame()
for file in pools_seller_pct_files:
    one_month_issue_pools = pd.read_csv(file)
    if len(seller_df)==0:
        seller_df = one_month_issue_pools
    else:
        if all(seller_df.columns == one_month_issue_pools.columns):
            seller_df = pd.concat([seller_df,one_month_issue_pools])
        else:
            print(f'columns missmatch for {file}')
    #print('Done loading ' + file)

In [10]:
seller_df.columns

Index(['poolno', 'asofdate', 'SellerPct_AMRHT', 'SellerPct_ALS',
       'SellerPct_CAFULL', 'SellerPct_CNTL', 'SellerPct_CITIZ', 'SellerPct_53',
       'SellerPct_FIR', 'SellerPct_FRDOM', 'SellerPct_GUILD',
       'SellerPct_CHASE', 'SellerPct_LLSL', 'SellerPct_MATRX', 'SellerPct_NCM',
       'SellerPct_NATIONST', 'SellerPct_NRESM', 'SellerPct_PNYMAC',
       'SellerPct_PILOSI', 'SellerPct_QUICK', 'SellerPct_REG',
       'SellerPct_RMSC', 'SellerPct_UNSHFI', 'SellerPct_WFHM', 'cusip',
       'Prefix', 'Label'],
      dtype='object')

In [11]:
print(f'seller_df.shape = {seller_df.shape}')

seller_df.shape = (225957, 27)


In [12]:
print(f'attr_df.shape = {attr_df.shape}')

attr_df.shape = (225957, 59)


In [13]:
print(f'geo_df.shape = {geo_df.shape}')

geo_df.shape = (225957, 58)


In [14]:
attr_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225957 entries, 0 to 53
Data columns (total 59 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   poolno             225957 non-null  object 
 1   asofdate           225957 non-null  int64  
 2   cusip              225957 non-null  object 
 3   Prefix             225957 non-null  object 
 4   spread             225455 non-null  float64
 5   Cpr1               225886 non-null  float64
 6   Cpr3               219323 non-null  float64
 7   Cpr6               209734 non-null  float64
 8   Cpr12              191420 non-null  float64
 9   Cpr24              158372 non-null  float64
 10  CprLife            225455 non-null  float64
 11  SMM                225945 non-null  float64
 12  DayCount           225957 non-null  int64  
 13  OBal               225957 non-null  int64  
 14  CBal               225455 non-null  float64
 15  pbal               225957 non-null  float64
 16  paydow

In [15]:
attr_df.drop(['Prefix','Cpr1','Cpr3','Cpr6','Cpr12','Cpr24','CprLife','pbal','paydown','Prepay','OCoupon','OWac','PCNLoans','PPNLoans','OSato','%cltv_80','%cltv_105','%cltv_125','%ccltv_80','%ccltv_105','%ccltv_125','%FedHold','%CMOHold','dti','codti'],axis=1,inplace=True)

In [16]:
attr_df.dropna(subset=['CBal','SMM','Wac','aols','fico','OccPct_investor','OccPct_owner','PropUnitsPct_2-4'],inplace=True)

In [17]:
attr_df.sort_values(by=['poolno','asofdate'],inplace=True)

In [18]:
attr_df.fillna(method='ffill',inplace=True)

In [19]:
attr_df['CBal'] = attr_df['CBal'].apply(lambda x: float(x))

In [20]:
attr_df.rename(columns={'Label':'pool_issue_month'},inplace=True)

In [21]:
attr_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225138 entries, 349 to 38
Data columns (total 34 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   poolno             225138 non-null  object 
 1   asofdate           225138 non-null  int64  
 2   cusip              225138 non-null  object 
 3   spread             225138 non-null  float64
 4   SMM                225138 non-null  float64
 5   DayCount           225138 non-null  int64  
 6   OBal               225138 non-null  int64  
 7   CBal               225138 non-null  float64
 8   factor             225138 non-null  float64
 9   Coupon             225138 non-null  float64
 10  Wac                225138 non-null  float64
 11  Wam                225138 non-null  int64  
 12  Age                225138 non-null  int64  
 13  aols               225138 non-null  float64
 14  waols              225138 non-null  float64
 15  ONLoans            225138 non-null  float64
 16  cnlo

In [22]:
geo_df.drop('Label',axis=1,inplace=True)

In [23]:
geo_df.sort_values(by=['poolno','asofdate'],inplace=True)

In [24]:
geo_df.fillna(method='ffill',inplace=True)

In [25]:
geo_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225957 entries, 348 to 38
Data columns (total 57 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   poolno       225957 non-null  object 
 1   asofdate     225957 non-null  int64  
 2   cusip        225957 non-null  object 
 3   StatePct_AK  225957 non-null  float64
 4   StatePct_AL  225957 non-null  float64
 5   StatePct_AR  225957 non-null  float64
 6   StatePct_AZ  225957 non-null  float64
 7   StatePct_CA  225957 non-null  float64
 8   StatePct_CO  225957 non-null  float64
 9   StatePct_CT  225957 non-null  float64
 10  StatePct_DC  225957 non-null  float64
 11  StatePct_DE  225957 non-null  float64
 12  StatePct_FL  225957 non-null  float64
 13  StatePct_GA  225957 non-null  float64
 14  StatePct_GU  225957 non-null  float64
 15  StatePct_HI  225957 non-null  float64
 16  StatePct_IA  225957 non-null  float64
 17  StatePct_ID  225957 non-null  float64
 18  StatePct_IL  225957 non-nu

In [26]:
geo_columns = ['StatePct_AK','StatePct_AL','StatePct_AR','StatePct_AZ','StatePct_CA','StatePct_CO','StatePct_CT',
                'StatePct_DC','StatePct_DE','StatePct_FL','StatePct_GA','StatePct_GU','StatePct_HI','StatePct_IA',
                'StatePct_ID','StatePct_IL','StatePct_IN','StatePct_KS','StatePct_KY','StatePct_LA','StatePct_MA',
                'StatePct_MD','StatePct_ME','StatePct_MI','StatePct_MN','StatePct_MO','StatePct_MS','StatePct_MT',
                'StatePct_NC','StatePct_ND','StatePct_NE','StatePct_NH','StatePct_NJ','StatePct_NM','StatePct_NV',
                'StatePct_NY','StatePct_OH','StatePct_OK','StatePct_OR','StatePct_PA','StatePct_PR','StatePct_RI',
                'StatePct_SC','StatePct_SD','StatePct_TN','StatePct_TX','StatePct_UT','StatePct_VA','StatePct_VI',
                'StatePct_VT','StatePct_WA','StatePct_WI','StatePct_WV','StatePct_WY']

In [27]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

In [28]:
geo_df[geo_columns] = geo_df[geo_columns].applymap(lambda x: float(x) if isfloat(x) else np.nan)

In [29]:
geo_df.fillna(0,inplace=True)

In [30]:
geo_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225957 entries, 348 to 38
Data columns (total 57 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   poolno       225957 non-null  object 
 1   asofdate     225957 non-null  int64  
 2   cusip        225957 non-null  object 
 3   StatePct_AK  225957 non-null  float64
 4   StatePct_AL  225957 non-null  float64
 5   StatePct_AR  225957 non-null  float64
 6   StatePct_AZ  225957 non-null  float64
 7   StatePct_CA  225957 non-null  float64
 8   StatePct_CO  225957 non-null  float64
 9   StatePct_CT  225957 non-null  float64
 10  StatePct_DC  225957 non-null  float64
 11  StatePct_DE  225957 non-null  float64
 12  StatePct_FL  225957 non-null  float64
 13  StatePct_GA  225957 non-null  float64
 14  StatePct_GU  225957 non-null  float64
 15  StatePct_HI  225957 non-null  float64
 16  StatePct_IA  225957 non-null  float64
 17  StatePct_ID  225957 non-null  float64
 18  StatePct_IL  225957 non-nu

In [31]:
seller_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225957 entries, 0 to 53
Data columns (total 27 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   poolno              225957 non-null  object 
 1   asofdate            225957 non-null  int64  
 2   SellerPct_AMRHT     225455 non-null  float64
 3   SellerPct_ALS       225455 non-null  float64
 4   SellerPct_CAFULL    225455 non-null  float64
 5   SellerPct_CNTL      225455 non-null  float64
 6   SellerPct_CITIZ     225455 non-null  float64
 7   SellerPct_53        225455 non-null  float64
 8   SellerPct_FIR       225455 non-null  float64
 9   SellerPct_FRDOM     225455 non-null  float64
 10  SellerPct_GUILD     225455 non-null  float64
 11  SellerPct_CHASE     225455 non-null  float64
 12  SellerPct_LLSL      225455 non-null  float64
 13  SellerPct_MATRX     225455 non-null  float64
 14  SellerPct_NCM       225455 non-null  float64
 15  SellerPct_NATIONST  225455 non-null  f

In [32]:
seller_df.drop(['Label','Prefix'],axis=1,inplace=True)

In [33]:
seller_df.sort_values(by=['poolno','asofdate'],inplace=True)

In [34]:
seller_df.fillna(method='ffill',inplace=True)

In [35]:
seller_columns = ['SellerPct_AMRHT','SellerPct_ALS','SellerPct_CAFULL','SellerPct_CNTL','SellerPct_CITIZ',
                  'SellerPct_53','SellerPct_FIR','SellerPct_FRDOM','SellerPct_GUILD','SellerPct_CHASE',
                  'SellerPct_LLSL','SellerPct_MATRX','SellerPct_NCM','SellerPct_NATIONST','SellerPct_NRESM',
                  'SellerPct_PNYMAC','SellerPct_PILOSI','SellerPct_QUICK','SellerPct_REG','SellerPct_RMSC',
                  'SellerPct_UNSHFI','SellerPct_WFHM']

In [36]:
seller_df[seller_columns] = seller_df[seller_columns].applymap(lambda x: float(x) if isfloat(x) else np.nan)

In [37]:
seller_df.fillna(0,inplace=True)

In [38]:
seller_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225957 entries, 348 to 38
Data columns (total 25 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   poolno              225957 non-null  object 
 1   asofdate            225957 non-null  int64  
 2   SellerPct_AMRHT     225957 non-null  float64
 3   SellerPct_ALS       225957 non-null  float64
 4   SellerPct_CAFULL    225957 non-null  float64
 5   SellerPct_CNTL      225957 non-null  float64
 6   SellerPct_CITIZ     225957 non-null  float64
 7   SellerPct_53        225957 non-null  float64
 8   SellerPct_FIR       225957 non-null  float64
 9   SellerPct_FRDOM     225957 non-null  float64
 10  SellerPct_GUILD     225957 non-null  float64
 11  SellerPct_CHASE     225957 non-null  float64
 12  SellerPct_LLSL      225957 non-null  float64
 13  SellerPct_MATRX     225957 non-null  float64
 14  SellerPct_NCM       225957 non-null  float64
 15  SellerPct_NATIONST  225957 non-null 

# Below we are combining the three DataFrames (attr_df, geo_df, seller_df) into a single DataFrame called df

In [39]:
print(f'attr_df.shape   = {attr_df.shape}')
print(f'geo_df.shape    = {geo_df.shape}')
print(f'seller_df.shape = {seller_df.shape}')

attr_df.shape   = (225138, 34)
geo_df.shape    = (225957, 57)
seller_df.shape = (225957, 25)


In [40]:
geo_df.drop('poolno',axis=1,inplace=True)

In [41]:
df = pd.merge(attr_df,geo_df,on=['cusip','asofdate'],how='left')

In [42]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225138 entries, 0 to 225137
Data columns (total 88 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   poolno             225138 non-null  object 
 1   asofdate           225138 non-null  int64  
 2   cusip              225138 non-null  object 
 3   spread             225138 non-null  float64
 4   SMM                225138 non-null  float64
 5   DayCount           225138 non-null  int64  
 6   OBal               225138 non-null  int64  
 7   CBal               225138 non-null  float64
 8   factor             225138 non-null  float64
 9   Coupon             225138 non-null  float64
 10  Wac                225138 non-null  float64
 11  Wam                225138 non-null  int64  
 12  Age                225138 non-null  int64  
 13  aols               225138 non-null  float64
 14  waols              225138 non-null  float64
 15  ONLoans            225138 non-null  float64
 16  cn

In [43]:
seller_df.drop('poolno',axis=1,inplace=True)

In [44]:
df = pd.merge(df,seller_df,on=['cusip','asofdate'],how='left')

In [45]:
df.fillna(0,inplace=True)

In [46]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225138 entries, 0 to 225137
Data columns (total 110 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   poolno              225138 non-null  object 
 1   asofdate            225138 non-null  int64  
 2   cusip               225138 non-null  object 
 3   spread              225138 non-null  float64
 4   SMM                 225138 non-null  float64
 5   DayCount            225138 non-null  int64  
 6   OBal                225138 non-null  int64  
 7   CBal                225138 non-null  float64
 8   factor              225138 non-null  float64
 9   Coupon              225138 non-null  float64
 10  Wac                 225138 non-null  float64
 11  Wam                 225138 non-null  int64  
 12  Age                 225138 non-null  int64  
 13  aols                225138 non-null  float64
 14  waols               225138 non-null  float64
 15  ONLoans             225138 non-nu

In [47]:
print(f'df.shape = {df.shape}')

df.shape = (225138, 110)


In [48]:
df.to_pickle('../jumbos_30yr.pkl')

In [50]:
# Write df into a sequence of small files

#out_dir = '../30y fixed rate data/fnm/conforming/clean data'
out_dir = '../30y fixed rate data/fnm/jumbos/clean data'

print(f'df.shape = {df.shape}')

rows = df.shape[0]

N = rows//30000 + 1

for i in range(N):
    df.iloc[i*30000:(i+1)*30000,].to_csv(f'{out_dir}/pools_part{i+1}.csv',index=False)

df.shape = (225138, 110)
